In [1]:
from PIL import Image, ImageOps
import torch
from torch import Tensor, linalg
import numpy as np
from IPython.display import Latex
import matplotlib.pyplot as plt
import os

# Generate the Data Matrix and the Label vector
Read data into torch.Tensor

In [ ]:
torch.set_default_dtype(torch.float64)

image_count_per_person = 10
person_count = 40
images = [Image.open(f"../data/s{j + 1}/{i + 1}.pgm") for j in range(person_count) for i in range(image_count_per_person)]

all_data = np.array(images)
all_data.resize((image_count_per_person * person_count, images[0].width * images[0].height))
all_data = Tensor(all_data)

labels = torch.tensor([i for i in range(person_count) for _ in range(image_count_per_person)])

# Split the Dataset into Training and Test sets
Split dataset into training and test data taking the even indexed rows for testing and the odd indexed rows for training

In [ ]:
odd_indices = [i for i in range(len(all_data)) if i % 2 == 1]
even_indices = [i for i in range(len(all_data)) if i % 2 == 0]

training_data = all_data[odd_indices]
test_data = all_data[even_indices]

training_labels = labels[odd_indices]
test_labels = labels[even_indices]